In [1]:
# ===============================================================
# Multi-Class DR Classification using Fuzzy SVM
# Dataset: Folder-based (train/0–4, test/0–4)
# ===============================================================

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

# ==============================================
# ✅ Step 1: Load images from train/test subfolders (0–4)
# ==============================================
train_dir = "/kaggle/input/diabetic-retinopathy-dataset/training_data_set/training_data_set"   # 🔹 change path
test_dir  = "/kaggle/input/diabetic-retinopathy-dataset/testing_dataset/testing_dataset"    # 🔹 change path

def load_dataset(base_dir):
    image_paths = []
    labels = []
    for label in sorted(os.listdir(base_dir)):
        class_folder = os.path.join(base_dir, label)
        if os.path.isdir(class_folder):
            for file in os.listdir(class_folder):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(class_folder, file))
                    labels.append(int(label))
    return image_paths, labels

train_image_paths, y_train = load_dataset(train_dir)
test_image_paths, y_test = load_dataset(test_dir)

print(f"✅ Train images: {len(train_image_paths)} | Test images: {len(test_image_paths)}")

# ==============================================
# ✅ Step 2: Load and preprocess all images
# ==============================================
def load_images(image_paths, target_size=(224, 224)):
    images = []
    for path in tqdm(image_paths, desc="Loading images"):
        img = image.load_img(path, target_size=target_size)
        img_array = image.img_to_array(img)
        images.append(preprocess_input(img_array))
    return np.array(images)

X_train_images = load_images(train_image_paths)
X_test_images = load_images(test_image_paths)

print("✅ Image data prepared successfully!")
print("Train shape:", X_train_images.shape)
print("Test shape :", X_test_images.shape)
print("Labels     :", np.unique(y_train))

# ==============================================
# ✅ Step 3: Feature Extraction (EfficientNetB0)
# ==============================================
try:
    print("⚙️ Extracting features using EfficientNetB0 (ImageNet weights)...")
    base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
except Exception:
    print("⚠️ ImageNet weights unavailable, switching to random init.")
    base_model = EfficientNetB0(weights=None, include_top=False, pooling='avg')

features_train = base_model.predict(X_train_images, batch_size=16, verbose=1)
features_test = base_model.predict(X_test_images, batch_size=16, verbose=1)

np.save("/kaggle/working/features_train.npy", features_train)
np.save("/kaggle/working/features_test.npy", features_test)
np.save("/kaggle/working/y_train.npy", y_train)
np.save("/kaggle/working/y_test.npy", y_test)
print("✅ Features extracted & saved!")

# ==============================================
# ✅ Step 4: Standardization
# ==============================================
scaler = StandardScaler()
X_train = scaler.fit_transform(features_train)
X_test = scaler.transform(features_test)

# ==============================================
# ✅ Step 5: Class Weights (Handle Imbalance)
# ==============================================
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print("Computed class weights:", class_weight_dict)

# ==============================================
# ✅ Step 8: Fuzzy SVM (Enhanced)
# ==============================================
print("\n⚙️ Computing fuzzy memberships for Fuzzy SVM...")
fuzzy_memberships = np.zeros_like(y_train, dtype=float)
for cls in np.unique(y_train):
    cls_idx = np.where(y_train == cls)[0]
    cls_mean = np.mean(X_train[cls_idx], axis=0)
    distances = np.linalg.norm(X_train[cls_idx] - cls_mean, axis=1)
    fuzzy_memberships[cls_idx] = 1 / (1 + distances)

# Normalize fuzzy memberships
fuzzy_memberships = fuzzy_memberships / np.max(fuzzy_memberships)

print("\n===== Fuzzy SVM Classifier =====")
svm_fuzzy = SVC(kernel='rbf', C=5, gamma='scale', probability=True)
svm_fuzzy.fit(X_train, y_train, sample_weight=fuzzy_memberships)
y_pred_fuzzy = svm_fuzzy.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_fuzzy))
print(classification_report(y_test, y_pred_fuzzy))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_fuzzy))

# ==============================================
# ✅ Step 9: Summary
# ==============================================
print("\n========== MODEL PERFORMANCE SUMMARY ==========")
print(f"Fuzzy SVM   : {accuracy_score(y_test, y_pred_fuzzy):.4f}")

print("\n✅ All results saved in /kaggle/working/")


2025-11-06 08:32:14.267613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762417934.854623      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762417935.009870      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Train images: 2500 | Test images: 496


Loading images: 100%|██████████| 496/496 [00:05<00:00, 96.21it/s] 


✅ Image data prepared successfully!
Train shape: (2500, 224, 224, 3)
Test shape : (496, 224, 224, 3)
Labels     : [0 1 2 3 4]
⚙️ Extracting features using EfficientNetB0 (ImageNet weights)...


2025-11-06 08:33:09.948343: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 137s 842ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 643ms/step
✅ Features extracted & saved!
Computed class weights: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}

⚙️ Computing fuzzy memberships for Fuzzy SVM...

===== Fuzzy SVM Classifier =====
Accuracy: 0.9657258064516129
              precision    recall  f1-score   support

           0       0.88      1.00      0.93       100
           1       0.98      0.86      0.91       100
           2       0.99      1.00      1.00       100
           3       1.00      0.98      0.99        96
           4       1.00      0.99      0.99       100

    accuracy                           0.97       496
   macro avg       0.97      0.97      0.97       496
weighted avg       0.97      0.97      0.97       496

Confusion Matrix:
 [[100   0   0   0   0]
 [ 14  86   0   0   0]
 [  0   0 100   0   0]
 [  0   2   0  94   0]
 [  0   0   1   0  99]]

========== MODEL PERFORMANCE S

In [2]:
import joblib

joblib.dump(svm_fuzzy, "fuzzy_svm_model.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [3]:
# ===============================================================
# Gradio Interface for Multi-Class DR Classification
# Using: Fuzzy SVM + EfficientNetB0 features
# ===============================================================

import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from sklearn.preprocessing import StandardScaler

# -------------------------
# Load models and scalers
# -------------------------
import joblib

# Load saved models (make sure to save them after training)
svm_fuzzy = joblib.load("/kaggle/working/fuzzy_svm_model.pkl")
scaler = joblib.load("/kaggle/working/scaler.pkl")

# Load feature extractor
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

# Class names
class_names = ['0', '1', '2', '3', '4']

# -------------------------
# Prediction function
# -------------------------
def predict_dr(img):
    # Preprocess image
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # Extract features
    features = base_model.predict(img_array)
    features_scaled = scaler.transform(features)
    
    # Predict with all models
    fuzzy_pred = svm_fuzzy.predict(features_scaled)[0]
    fuzzy_conf = svm_fuzzy.predict_proba(features_scaled)[0]
    
    # Prepare results
    results = {
        "Fuzzy SVM Prediction": fuzzy_pred,
        "Fuzzy SVM Confidence": {class_names[i]: float(fuzzy_conf[i]) for i in range(5)}
    }
    
    return results

# -------------------------
# Gradio Interface
# -------------------------
interface = gr.Interface(
    fn=predict_dr,
    inputs=gr.Image(type="pil"),
    outputs=gr.JSON(),
    title="Diabetic Retinopathy Multi-Class Prediction",
    description="Upload a retinal fundus image and get predictions & confidence from Fuzzy SVM models."
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://1752c6c57745d92606.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
